In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
import io
from PIL import Image
from time import sleep
import time
import random
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import os
import pandas as pd
import numpy as np


In [ ]:
# Declare browser
s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

# Khởi tạo trình duyệt Chrome với tùy chọn toàn màn hình
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=s, options=chrome_options)

# Open URL
# driver.get("https://www.thegioididong.com/dtdd#c=42&o=17&pi=5")
driver.get("https://www.thegioididong.com/laptop#c=44&o=17&pi=14")
sleep(10)

# Scroll down
# for i in range(5):
#     scroll_script = "window.scrollBy(0, window.innerHeight);"
#     driver.execute_script(scroll_script)
#     time.sleep(1) 

# Scroll up
for i in range(70):
    scroll_script = "window.scrollBy(0, -window.innerHeight);"
    driver.execute_script(scroll_script)
    time.sleep(1)
    

# elems2 = driver.find_elements(By.CLASS_NAME , "jBwCF ")
# links = [elem.get_attribute('src') for elem in elems2 if elem.get_attribute('type') == 'product' and elem.get_attribute('src').count("data:image") <=0]

# ================================ GET id
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_44 a')
id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]

# ================================ GET title
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_44 a')
title = [elem.get_attribute('data-name') for elem in elems if elem.get_attribute('data-name') is not None]

# ================================ GET img_thumb
elems = driver.find_elements(By.CSS_SELECTOR, '.item-img.item-img_44 img')
link_img_thumb = [elem.get_attribute('src') for elem in elems if elem.get_attribute('src') is not None and "Products" in elem.get_attribute('src')]


# ================================ GET href
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_44 a')
href = [elem.get_attribute('href') for i, elem in enumerate(elems) if i % 3 == 0]

# ================================ GET price
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_44 a')
price = [elem.get_attribute('data-price') for elem in elems if elem.get_attribute('data-price') is not None]

# Clean price
price = [int(float(number)) for number in price]

# ================================ GET price pre discount
discount_list, discount_idx, discount_percent_list = [], [], []
discount_list, discount_idx, discount_percent_list = [], [], []
for i in range(1, len(title)+1):
    try:

        try:
            discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[5]/span".format(i))
        except NoSuchElementException:
            discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[4]/span".format(i))

            
        try:
            discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[4]/p".format(i))
        except NoSuchElementException:
            discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[5]/p".format(i))

        discount_list.append(discount.text)

        discount_percent_list.append(discount_percent.text)
        print(i)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))

# ================================ GET brand
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_44 a')
brand = [elem.get_attribute('data-brand') for elem in elems if elem.get_attribute('data-brand') is not None]

# for elem in elems:
# 	_href = 'thegioididong.com' + elem.get_attribute('href')
# 	href.append(_href)

# Close browser
driver.close() 

In [31]:
for i, number in enumerate(discount_list):
    if discount_list[i] and discount_list[i] != '':
        # Kiểm tra xem chuỗi có giá trị hợp lệ
        if '.' in discount_list[i]:
            discount_list[i] = int(discount_list[i].replace('.', '').replace('₫', ''))

In [32]:
df1 = pd.DataFrame(list(zip(id_product, title, brand ,href ,link_img_thumb , price)), columns = ['id', 'title' ,'brand', 'href', 'link_img_thumb', 'price'])
# Đặt tên tệp Excel và sheet
excel_file = 'laptop.xlsx'
sheet_name = 'Sheet1'

df1['index_']= np.arange(1, len(df1) + 1)

df2 = pd.DataFrame(list(zip(discount_list ,discount_percent_list, discount_idx)), columns = ['discount', 'discount_percent','discount_idx'])
df3 = df1.merge(df2, how='left', left_on='index_', right_on='discount_idx')

# Lưu DataFrame vào tệp Excel
df3.to_excel(excel_file, sheet_name=sheet_name, index=False, engine='openpyxl')

In [27]:
len(link_img_thumb)

283

In [28]:
link_img_thumb

['https://cdn.tgdd.vn/Products/Images/44/231244/macbook-air-m1-2020-gray-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/313084/hp-15s-fq5229tu-i3-8u237pa-thumb-600x600.png',
 'https://cdn.tgdd.vn/Products/Images/44/304867/asus-tuf-gaming-f15-fx506hf-i5-hn014w-thumb-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/314630/acer-aspire-5-a515-58gm-51lb-i5-nxkq4sv002-170923-015941-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/311178/asus-vivobook-go-15-e1504fa-r5-nj776w-thumb-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/305600/acer-aspire-3-a315-58-589k-i5-nxam0sv008-thumb-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/311114/hp-240-g9-i3-6l1x7pa-thumb-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/309375/asus-vivobook-16-x1605va-i5-mb360w-thumb-laptop-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/302532/hp-15s-fq5162tu-i5-7c134pa-thumb-600x600.jpg',
 'https://cdn.tgdd.vn/Products/Images/44/312414/asus-vivobook-15-x1504za-i3-nj102w-t

In [3]:
def download_image(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{title[i]}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


urls = img_thumb

dowloadPath = r'C:\Users\ngoct\Downloads\Image-Scraper-And-Downloader-main\imgs'
for i, url in enumerate(urls):
	download_image(dowloadPath, url, i)

Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
